## Installs

In [ ]:
!pip3 install --upgrade --quiet langchain langchain-community langchain-openai chromadb 
!pip3 install --upgrade --quiet pypdf pandas streamlit python-dotenv

## Imports

In [1]:
# Import Langchain modules
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Optional

# Other modules and packages
import os
import tempfile
import streamlit as st  
import pandas as pd
from dotenv import load_dotenv
import uuid

## Env variables

In [2]:
load_dotenv()
OPEN_AI_API_KEY = os.environ.get("OPEN_AI_API_KEY")

## Process PDF documents

### Find and load all PDF documents

In [3]:
documents = []

# Traverse the directory tree and collect specified file in the ./data folder
loader = PyPDFLoader("data/Ubisoft_FY25_Q1_Sales_EN_vFinal.pdf")
documents=loader.load()

### Split documents

In [ ]:
# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, 
                                               chunk_overlap=120, 
                                               length_function=len, 
                                               separators=["\n\n", "\n", " "])

documents = text_splitter.split_documents(documents)

### Define embedding function

In [5]:
# Use embedding function from OpenAI
embedding_function = embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002", openai_api_key=OPEN_AI_API_KEY
    )